In [1]:
# import libraries we need
%pylab inline
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
tfd = tfp.distributions

import sys
sys.path.append('./../py_programs/')
from Simulator import *
from keract import get_activations

Populating the interactive namespace from numpy and matplotlib


### load data from files

In [2]:
# load the data from files
filenames = []
x_ = []
y_ = []

# iterate over all files
#filenames = ['./../simulation/data/laser'] + ['./../simulation/data/sps']
filenames = ['./../simulation/data/halfhalf_rand/'+str(i) for i in range(50)]


for filename in filenames: 
    data = load_data(filename=filename)
    x_.append(data)
    
x_ = np.array(x_)

In [3]:
# convert data into correct shape
x_ = x_.reshape(x_.shape[0]*x_.shape[1],x_.shape[2],x_.shape[3])

# get the ground truth
threshold = 0.5
y_ = get_truth(x_, thr=threshold)

# we only need the g2signal for training
x_ = x_[:,2,:]

Now we've collected all the datasets. For training the model, we need to **shuffle** all data sets first


In [33]:
# shuffle
i = np.random.permutation(len(x_))
x_ = x_[i]
y_ = y_[i]

# reshape
x_ = x_.reshape(x_.shape[0], x_.shape[1], 1)
y_ = y_.reshape(y_.shape[0])

# allocate
x_train = x_[100:]
y_train = y_[100:]
x_test = x_[:100]
y_test = y_[:100]

In [5]:
# check the sps rate (data sets considered as sps out of all data sets) 
print(y_train[y_train==1].shape[0]/y_train.shape[0])
print(y_test[y_test==1].shape[0]/y_test.shape[0])

0.5859183673469388
0.54


### get test data

In [47]:
# load the data from files
filenames = []
x_test = []
y_test = []

# iterate over all files
filenames = ['./../simulation/data/test_data/'+str(i) for i in range(20)]
#filenames = ['./../simulation/data/halfhalf_rand/'+str(i) for i in range(50)]

#filenames = ['./../simulation/data/sps'] + ['./../simulation/data/laser']


for filename in filenames: 
    data = load_data(filename=filename)
    x_test.append(data)
    
x_test = np.array(x_test)
    

In [48]:
# convert simulated data into correct shape
x_test = x_test.reshape(x_test.shape[0]*x_test.shape[1],x_test.shape[2],x_test.shape[3])

# get the ground truth
threshold = 0.5
y_test = get_truth(x_test, thr=threshold)

# we only need the g2signal
x_test = x_test[:,1,:]


# shuffle
i = np.random.permutation(len(x_test))
x_test = x_test[i]
y_test = y_test[i]

# reshape
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)
y_test = y_test.reshape(y_test.shape[0])

---
# CNN model

In [8]:
def make_model(input_shape):
    # input shape should be (time signal, 1)
    input_layer = keras.layers.Input(input_shape)

    # 1st con1d layer
    conv1 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(input_layer)
    
    # 2nd con1d layer
    conv2 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv1)

    # 3rd con1d layer
    conv3 = keras.layers.Conv1D(filters=64, kernel_size=4, padding='valid', strides=1, activation='relu')(conv2)
    
    # maxpooling layer
    pool = keras.layers.MaxPool1D(pool_size=3, strides=2, padding='valid')(conv3) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    # flatten layer
    flat = keras.layers.Flatten()(pool)
    
    # fully connected layer to output a binary vector
    dense1 = keras.layers.Dense(2, activation='relu')(flat)
    #dense2 = keras.layers.Dense(2, activation='relu')(dense1)

    return keras.models.Model(inputs=input_layer, outputs=dense1)


model = make_model(input_shape=x_test.shape[1:])


# currently working on

In [35]:
# current working on
def make_model2(input_shape):
    # input shape should be (vinvalues, 1)
    input_layer = keras.layers.Input(input_shape)
     
    lam1 = keras.layers.Lambda(lambda x: -x)(input_layer)
    
    min1 = keras.layers.MaxPool1D(pool_size=4,strides=5, padding='same')(lam1)
    
    lam2 = keras.layers.Lambda(lambda x: -x)(min1)
    
    conv1 = keras.layers.Conv1D(filters=4, kernel_size=3, padding='same', strides=1)(lam2)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    # maxpooling layer
    #pool = keras.layers.MaxPool1D(pool_size=5, strides=5, padding='same')(conv1) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    # flatten layer
    flat = keras.layers.Flatten()(conv1)
    
    #pool = keras.layers.GlobalAveragePooling1D()(conv1) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    
    # fully connected layer to output a binary vector
    #dense1 = keras.layers.Dense(10, activation='relu')(flat)
    dense2 = keras.layers.Dense(1, activation='sigmoid')(flat)

    return keras.models.Model(inputs=input_layer, outputs=dense2)


model = make_model2(input_shape=x_.shape[1:])


In [ ]:
def make_model3(input_shape):
    # input shape should be (time signal, 1)
    input_layer = keras.layers.Input(input_shape)
     
    #lam1 = keras.layers.Lambda(lambda x: -x)(input_layer)
    
    #min1 = keras.layers.MaxPool1D(pool_size=4,strides=2,padding='same')(lam1)
    
    #lam2 = keras.layers.Lambda(lambda x: -x)(min1)
    
    conv1 = keras.layers.Conv1D(filters=1, kernel_size=5, padding='same', strides=1, activation='relu')(input_layer)
    
    # maxpooling layer
    #pool = keras.layers.MaxPool1D(pool_size=5, strides=5, padding='same')(lam2) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    # flatten layer
    #flat = keras.layers.Flatten()(input_layer)
    
    # fully connected layer to output a binary vector
    dense1 = keras.layers.Dense(2, activation='relu')(conv1)
    #dense2 = keras.layers.Dense(2, activation='relu')(dense1)

    return keras.models.Model(inputs=input_layer, outputs=dense1)


model = make_model2(input_shape=x_.shape[1])


In [ ]:
def make_model4(input_shape):
    # input shape should be (time signal, 1)
    input_layer = keras.layers.Input(input_shape)
     
    lam1 = keras.layers.Lambda(lambda x: -x)(input_layer)
    
    min1 = keras.layers.MaxPool1D(pool_size=4,strides=5, padding='same')(lam1)
    
    lam2 = keras.layers.Lambda(lambda x: -x)(min1)
    
    conv1 = keras.layers.Conv1D(filters=4, kernel_size=3, padding='same', strides=1)(lam2)
    #conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)
    # maxpooling layer
    #pool = keras.layers.MaxPool1D(pool_size=5, strides=5, padding='same')(conv1) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    # flatten layer
    flat = keras.layers.Flatten()(conv1)
    
    #pool = keras.layers.GlobalAveragePooling1D()(conv1) # keras.layers.GlobalAveragePooling1D()(conv3) #  # 
    
    
    # fully connected layer to output a binary vector
    #dense1 = keras.layers.Dense(10, activation='relu')(flat)
    #dense2 = keras.layers.Dense(1, activation='sigmoid')(flat)
    dense2 = keras.layers.Dense(1)(flat)
    dense2 = keras.layers.ReLU(max_value=)(dense2)

    return keras.models.Model(inputs=input_layer, outputs=dense2)


model = make_model2(input_shape=x_.shape[1:])


In [28]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 200, 1)]          0         
_________________________________________________________________
lambda_7 (Lambda)            (None, 200, 1)            0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 40, 1)             0         
_________________________________________________________________
lambda_8 (Lambda)            (None, 40, 1)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 40, 4)             16        
_________________________________________________________________
batch_normalization_3 (Batch (None, 40, 4)             16        
_________________________________________________________________
re_lu_3 (ReLU)               (None, 40, 4)             0   

In [36]:
epochs = 30
batch_size = 5

callbacks = [
    # save checkpoints
    #keras.callbacks.ModelCheckpoint(
    #    "model2.h5", save_best_only=True, monitor="val_loss"
    #),
    
    # if there's no improvement for minimizing losses, which makes the training stagnate
    # then reduce the learning rate
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    
    # stop training if a monitored metric stops improving
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

model.compile(
    optimizer='adam',  #tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss='mse',
    metrics=['sparse_categorical_accuracy','binary_accuracy'],
)

history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.3,
    verbose=1,
)


Epoch 1/30
686/686 [==============================] - 1s 653us/step - loss: 0.1657 - sparse_categorical_accuracy: 0.4269 - binary_accuracy: 0.7581 - val_loss: 0.0622 - val_sparse_categorical_accuracy: 0.4041 - val_binary_accuracy: 0.9401
Epoch 2/30
686/686 [==============================] - 0s 525us/step - loss: 0.0493 - sparse_categorical_accuracy: 0.4074 - binary_accuracy: 0.9479 - val_loss: 0.0375 - val_sparse_categorical_accuracy: 0.4041 - val_binary_accuracy: 0.9837
Epoch 3/30
686/686 [==============================] - 0s 538us/step - loss: 0.0301 - sparse_categorical_accuracy: 0.4258 - binary_accuracy: 0.9797 - val_loss: 0.0265 - val_sparse_categorical_accuracy: 0.4041 - val_binary_accuracy: 0.9837
Epoch 4/30
686/686 [==============================] - 0s 515us/step - loss: 0.0238 - sparse_categorical_accuracy: 0.4292 - binary_accuracy: 0.9801 - val_loss: 0.0221 - val_sparse_categorical_accuracy: 0.4041 - val_binary_accuracy: 0.9837
Epoch 5/30
686/686 [============================

In [ ]:
model = keras.models.load_model('g2model.h5')


In [ ]:
model.compile(loss="mse", optimizer="adam",metrics=['sparse_categorical_accuracy','binary_accuracy'])
activations = get_activations(model, x_train, auto_compile=False)

In [ ]:
[print(k, '->', v.shape, '- Numpy array') for (k, v) in activations.items()]

In [ ]:
plt.plot(activations[list(activations.keys())[0]][0])
#plt.plot(activations["conv1d_5"][:,0,0])
#plt.plot(activations[list(activations.keys())[4]][1,:,0],label=list(activations.keys())[4])

#plt.plot(activations[list(activations.keys())[-1]][:,:],label=list(activations.keys())[-1])
#plt.plot(activations["max_pooling1d_46"][900,:,0])
#activations[list(activations.keys())[6]]
plt.legend(loc='best')

In [37]:
r=np.round(model.predict(x_train[:100]))
print(np.squeeze(r))
print(y_train[:100])

[1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 1.]
[1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 1. 1.]


In [38]:
model.evaluate(x_train,y_train)

154/154 [==============================] - 0s 622us/step - loss: 0.0059 - sparse_categorical_accuracy: 0.4151 - binary_accuracy: 0.9939


[0.005880405660718679, 0.4151020348072052, 0.9938775300979614]

In [49]:
r=np.round(model.predict(x_test)[:100])
print(np.squeeze(r))
print(y_test[:100])


[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
[0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.
 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0.
 0. 0. 1. 0.]


In [50]:
model.evaluate(x_test,y_test)

63/63 [==============================] - 0s 668us/step - loss: 0.3694 - sparse_categorical_accuracy: 0.4510 - binary_accuracy: 0.5490


[0.3693941533565521, 0.45100000500679016, 0.5490000247955322]

In [51]:
y_test[y_test==1].shape[0]/y_test.shape[0]

0.549

In [21]:
y_train[y_train==1].shape[0]/y_train.shape[0]

0.5859183673469388

In [ ]:
y_[:100]

In [ ]:
r=model.predict(x_[:100])
np.round(r.flatten())

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()


In [ ]:
plt.plot(x_train[0])

In [ ]:
y_train[0]

In [ ]:
plt.plot(x_train[1])

In [ ]:
np.loadtxt('./simulation/data/s1set10.txt', skiprows=9, max_rows=1)


In [ ]:
np.loadtxt('./simulation/data/s1set10.txt', skiprows=9, max_rows=1)
binnum = int(np.loadtxt('./simulation/data/s1set10.txt', skiprows=4, usecols=1, max_rows=1, delimiter=':'))
plt.plot(np.loadtxt('./simulation/data/s1set10.txt', skiprows=13, max_rows=binnum))


In [ ]:
binnum = int(np.loadtxt('./simulation/data/s1set10.txt', skiprows=4, usecols=1, max_rows=1, delimiter=':'))


In [ ]:
s1 = Simulator.simulator(gt=0.5, Nbins=200, width=1., Ndet=1e6, sps=1., laser=0., ther=0., non=0.)


In [ ]:
s1.get_data(s1.distribution)

In [ ]:
filenames = []
x_ = []
y_ = []
filenames = ['./simulation/data/s1set'+str(i)+'.txt' for i in range(500)]+['./simulation/data/s2set'+str(i)+'.txt' for i in range(500)]

for filename in filenames: 
    data = s2.load_data(filename)
    x_.append(data[1])
    y_.append(data[0])

In [ ]:
len(x_)

In [ ]:
# convert the keras model into TF lite
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


In [ ]:
model = keras.models.load_model('g2model.h5')


In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()


In [ ]:
model.trainable_weights

In [ ]:
import numpy as np


In [ ]:
binnum = np.arange(-100,100)
g2 = np.arange(200,400)
value = np.arange(400,600)
st = np.arange(30)
data = np.zeros((30,3,200))

In [ ]:
for i in st:
    data[i] = np.array([binnum,g2,value])

In [ ]:
for i in st:
    try:
        data[i] = np.array([signal,binnum,his])
    except:
        raise Exception('Input distribution is incorrect, please check the light sources fraction.')
        break
        

In [ ]:
filename = 'test2' + '.csv'
            
file = open(filename,'w')  # 'a' for appending
file.write('data array format: (N seperate measurements  [binnumber  normalized g2 signal  binvalues]  data) ')
file.write('\n')
np.savetxt(file, data[:,2,:], header='histogram values (int)') 
np.savetxt(file, data[:,1,:], header='normalized g2 signal') 
np.savetxt(file, data[:,0,:], header='bin number array') 
file.close()


In [ ]:
def load_data(self, filename):
        gt = np.loadtxt(filename, skiprows=1, max_rows=1)
        binnum = int(np.loadtxt(filename, skiprows=4, usecols=1, max_rows=1, delimiter=':'))
        g2 = np.loadtxt(filename, skiprows=12, max_rows=binnum)
        
        return [gt, g2]

In [ ]:
        parse = np.loadtxt(filename, skiprows=1, max_rows=1000000)
        data = np.zeros((parse.shape[0]//3, 3, parse.shape[1]))
        data[:,0,:] = parse[:parse.shape[0]//3,:]
        data[:,1,:] = parse[parse.shape[0]//3:parse.shape[0]*2//3,:]
        data[:,2,:] = parse[parse.shape[0]*2//3:,:]
        

In [ ]:
def load_data(filename):
    '''
    input: path + filename
    output: data sets, same format with generated data sets
    '''
    # I must say, this is a stupid way for parsing. I will switch to pandas once available
    parse = np.loadtxt(filename, skiprows=1, max_rows=1000000)
    data = np.zeros((parse.shape[0]//3, 3, parse.shape[1]))
    data[:,0,:] = parse[:parse.shape[0]//3,:]
    data[:,1,:] = parse[parse.shape[0]//3:parse.shape[0]*2//3,:]
    data[:,2,:] = parse[parse.shape[0]*2//3:,:]
    
    return data
data=load_data('./test2.csv')

In [ ]:
def get_truth(data,thr):
    '''
    input: data array, threshhold for classifying sps or not sps
    output: a binary result array
    '''
    signal = data[:,1,:]
    binnumber = data[:,2,:]
    
    # create a 1d-array of g2(0) values
    g2zero = np.ndarray.flatten(np.array([signal[i][binnumber[i]==0] for i in range(signal.shape[0])]))
    binary = np.zeros(g2zero.shape)
    
    # if it's smaller than the threshold then make it to 1 (it is a sps)
    binary[[g2zero[i]<thr for i in range(len(g2zero))]] = 1
    
    return binary

In [ ]:
    signal = data[:,1,:]
    binnumber = data[:,2,:]
    signal[:,binnumber==0.]

In [ ]:
    g2zero = np.ndarray.flatten(np.array([signal[i][binnumber[i]==0] for i in range(signal.shape[0])]))
    thr = 3000

In [ ]:
binary